In [1]:
from iwnlp.iwnlp_wrapper import IWNLPWrapper

In [2]:
lemmatizer = IWNLPWrapper(lemmatizer_path='C:/Users/1/Desktop/thesis/IWNLP.Lemmatizer_20170501.json')

In [3]:
def get_lemma(word):
    lemma = lemmatizer.lemmatize_plain(word)
    if lemma is None:
        return capitalize(word)
    else:
        return capitalize(lemma[0])

In [4]:
def capitalize(word):
    if len(word) > 1:
        word = word[0].upper() + word[1:]
    return word

In [5]:
file_words = open('C:/Users/1/Desktop/thesis/wortliste.txt', 'r', encoding='utf-8')
lines_words = file_words.readlines()

In [6]:
dictionary = []
for line in lines_words:
    line_new = line.split(' ')
    dictionary.append(capitalize(line_new[0]))

In [7]:
def is_in_dic(word):
    if word in dictionary:
        return True

In [8]:
def is_part(word):
    if len(word) > 1 and is_in_dic(word) is True:
        return True
    if (word.endswith('n') or word.endswith('e') or word.endswith('s')) and is_in_dic(word[:-1]):
        return True
    if (word.endswith('en') or word.endswith('er') or word.endswith('es')) and is_in_dic(word[:-2]):
        return True

In [9]:
def split_compound(word):
    max_ind = len(word)
    
    for ind, char in enumerate(word):
        left_compound = word[0:max_ind-ind]
        right_compound = word[max_ind-ind:max_ind]
        
        if is_part(left_compound) and len(left_compound) != len(word):
            right_compound_upper = capitalize(right_compound)
            if is_part(right_compound_upper):
                if not is_in_dic(left_compound) and is_in_dic(left_compound[:-1]):
                    return (left_compound[:-1], right_compound_upper)
                if not is_in_dic(left_compound) and is_in_dic(left_compound[:-2]):
                    return (left_compound[:-2], right_compound_upper)
                if is_in_dic(left_compound):
                    return (left_compound, right_compound_upper)

In [10]:
def split_compound_of_three(word):
    max_ind = len(word)
    
    for ind1 in range(max_ind):
        for ind2 in range(ind1, max_ind):
            left_part = word[:ind1]
            middle_part = word[ind1:ind2]
            right_part = word[ind2:]
            
            if is_part(left_part):
                middle_part_upper = capitalize(middle_part)
                right_part_upper = capitalize(right_part)
                if is_part(middle_part_upper) and is_part(right_part_upper):
                    if not is_in_dic(left_part) and is_in_dic(left_part[:-1]) and is_in_dic(middle_part_upper):
                        return (left_part[:-1], middle_part_upper, right_part_upper)
                    if not is_in_dic(left_part) and is_in_dic(left_part[:-2]) and is_in_dic(middle_part_upper):
                        return (left_part[:-2], middle_part_upper, right_part_upper)  
                    if is_in_dic(left_part) and not is_in_dic(middle_part_upper) and is_in_dic(middle_part_upper[:-1]):
                        return (left_part, middle_part_upper[:-1], right_part_upper)
                    if is_in_dic(left_part) and not is_in_dic(middle_part_upper) and is_in_dic(middle_part_upper[:-2]):
                        return (left_part, middle_part_upper[:-2], right_part_upper)
                    if is_in_dic(left_part) and is_in_dic(middle_part_upper):
                        return (left_part, middle_part_upper, right_part_upper)

In [11]:
s = open("C:/Users/1/Desktop/thesis/corpus_final/corpus_de.txt", "r", encoding="utf-8")
f = open("C:/Users/1/Desktop/thesis/corpus_final/corpus_de_splitted.txt", "w", encoding="utf-8")
w = open("C:/Users/1/Desktop/thesis/corpus_final/compounds_extracted.txt", "w", encoding="utf-8")

In [12]:
lines = s.readlines()
lines_new = []

for index in range(len(lines)):
    line = lines[index].strip("\n")
    tokens = line.split(" ")
    tokens_new = []
    for token in tokens:
        token_cap = capitalize(token)
        
        if split_compound(token_cap):
            comp = split_compound(token_cap)
            tokens_new.append(comp[0])
            tokens_new.append(comp[1])
            w.write(comp[0] + "/" + comp[1] + "/" + token + "/" + str(index) + "\n")
            
        if split_compound_of_three(token_cap):
            comp2 = split_compound_of_three(token_cap)
            tokens_new.append(comp2[0])
            tokens_new.append(comp2[1])
            tokens_new.append(comp2[2])
            w.write(comp2[0] + "/" + comp2[1] + "/" + comp2[2] + "/" + token + "/" + str(index) + "\n")
            
        if not split_compound(token) and not split_compound_of_three(token):
            tokens_new.append(token)
            
    line_new = " ".join(tokens_new)
    lines_new.append(line_new)
w.close()

for line in lines_new:
    f.write(line+"\n")
f.close()

s.close()